In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lpad, lit, when

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Aroon_Up/Aroon_Down") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Đọc dữ liệu từ bảng nguồn machungkhoan
df_lichsugia = spark.read.format("iceberg").load("stock_db.datn_lichsugia")
df_machungkhoan = spark.read.format("iceberg").load("stock_db.datn_machungkhoan")

# Convert 'ngay' to DateType
df_lichsugia = df_lichsugia.withColumn("ngay", to_date(col("ngay"), "dd/MM/yyyy"))

# Convert 'ky_du_lieu' to TimestampType
df_lichsugia = df_lichsugia.withColumn("ky_du_lieu", to_timestamp(col("ky_du_lieu"), "dd/MM/yyyy HH:mm:ss"))

# Extract numeric values from 'thaydoi' and split into change and percentage change
df_lichsugia = df_lichsugia.withColumn("thaydoi_value", regexp_extract(col("thaydoi"), r'([\d\.-]+)', 1).cast("float"))
df_lichsugia = df_lichsugia.withColumn("thaydoi_percent", regexp_extract(col("thaydoi"), r'\(([\d\.-]+)%\)', 1).cast("float"))

# Convert 'ngay' to 'dateid' in the format ddMMyyyy
df_lichsugia = df_lichsugia.withColumn("dateid", date_format(col("ngay"), "ddMMyyyy").cast("int"))

dim_stock_df = df_machungkhoan.filter((col('categoryname').isNotNull()) & (col('categoryname') != ""))
df_stock_drop_duplicate = dim_stock_df.dropDuplicates(['symbol'])

# Ghi dữ liệu vào bảng dim_stock
dim_stock_data = df_stock_drop_duplicate.select(
    col("symbol").alias("stocksymbol"),
    "companyname",
    "categoryid"
).distinct()

# Convert dim_stock_df to a list of stocksymbols
valid_stocksymbols = [row.stocksymbol for row in dim_stock_data.select("stocksymbol").distinct().collect()]

# Select and rename columns to match the schema of 'fact_price_history'
df_fact_price_history = df_lichsugia.select(
    col("symbol").alias("stocksymbol"),
    col("dateid"),
    col("giamocua").alias("openprice").cast("decimal(18, 2)"),
    col("giadongcua").alias("closeprice").cast("decimal(18, 2)"),
    col("giacaonhat").alias("highprice").cast("decimal(18, 2)"),
    col("giathapnhat").alias("lowprice").cast("decimal(18, 2)"),
    col("khoiluongkhoplenh").alias("volume").cast("bigint")
)

# Đảm bảo df_fact_price_history đã được lọc để loại bỏ các giá trị null
df_fact_price_history_filtered = df_fact_price_history.filter(col("stocksymbol").isin(valid_stocksymbols))

df_fact_price_history_filtered = df_fact_price_history_filtered.withColumn('dateid_padded', lpad(col('dateid').cast('string'), 8, '0'))
df_fact_price_history_filtered = df_fact_price_history_filtered.withColumn('date', to_date(col('dateid_padded'), 'ddMMyyyy'))

# Tính toán giá trị A/D cho từng ngày, xử lý trường hợp mẫu số bằng 0
df_with_ad_calculator = df_fact_price_history_filtered.withColumn(
    "A/D",
    when(
        (col("highprice") - col("lowprice")) != 0,
        (((col("closeprice") - col("lowprice")) - (col("highprice") - col("closeprice"))) * col("volume")) / (col("highprice") - col("lowprice"))
    ).otherwise(0)
)

# Định nghĩa indicatorid và chuẩn bị dữ liệu
indicator_data_ad = df_with_ad_calculator.select(
    col("stocksymbol"),
    col("dateid"),
    col("A/D").alias("indicatorvalue"),
    lit(2).alias("indicatorid")  # Giả sử 2 là ID cho A/D
)


In [16]:
# Ghi kết quả vào bảng `fact_stock_indicator`
indicator_data_ad.write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://10.168.6.106:3306/dtm_stock") \
    .option("dbtable", "fact_stock_indicator") \
    .option("user", "acc_etl") \
    .option("password", "Vnpt123456") \
    .mode("append") \
    .save()